# Mass balance of all glaciers: Maps

## Import packages

In [ ]:
import fsspec
import geopandas as gpd
import matplotlib.pyplot as plt
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
year_start = 1975
year_stop = 2020

## Define request

In [ ]:
collection_id = "derived-gridded-glacier-mass-change"
request = {
    "variable": "glacier_mass_change",
    "product_version": "wgms_fog_2022_09",
    "format": "zip",
    "hydrological_year": [
        f"{year}_{str(year + 1)[-2:]}" for year in range(year_start, year_stop + 1)
    ],
}

## Functions to cache

In [ ]:
def sum_and_add_trends(ds):
    da = ds["Glacier"]
    cumsum = da.cumsum("time", keep_attrs=True).drop("time")

    # Sum
    ds = ds.sum("time", keep_attrs=True)
    for da in ds.data_vars.values():
        da.attrs["long_name"] = f"Sum of {da.attrs['long_name']}"

    # Linear
    ds["Slope"] = cumsum.polyfit("time", 1)["polyfit_coefficients"].sel(degree=1)
    ds["Slope"].attrs = {
        "long_name": f"Trend of {da.attrs['long_name']}",
        "units": f"{da.attrs['units']} yr$^{-1}$",
    }

    # Quadratic
    ds["Acceleration"] = cumsum.polyfit("time", 2)["polyfit_coefficients"].sel(degree=2)
    ds["Acceleration"].attrs = {
        "long_name": f"Quadratic Trend of {da.attrs['long_name']}",
        "units": f"{da.attrs['units']} yr$^{-2}$",
    }

    return ds

## Download and transform data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"hydrological_year": 1},
    transform_func=sum_and_add_trends,
    transform_chunks=False,
)
# Customize some attributes
for da in ds.data_vars.values():
    da.attrs["long_name"] = da.attrs["long_name"].replace("_", " ").title()

## Create geodataframe

In [ ]:
# Maps
ds_stack = ds.stack(latlon=("latitude", "longitude"))
ds_stack = ds_stack.where((ds_stack["Glacier"] != 0).compute(), drop=True)
df = ds_stack.to_pandas()
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326",
)

## Plotting function

In [ ]:
def plot_map(gdf, var_name=None, label=None, title=None, **kwargs):
    kwargs = {"markersize": 5, "legend": var_name is not None} | kwargs
    if var_name:
        kwargs = {"c": var_name, "column": var_name} | kwargs
        kwargs.setdefault("legend_kwds", {"shrink": 0.49, "extend": "both"})
        if label is not None:
            kwargs["legend_kwds"].setdefault("label", label)

    shapefile_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip"
    with fsspec.open(f"simplecache::{shapefile_url}") as f:
        gdf_countries = gpd.read_file(f)

    ax = gdf_countries.boundary.plot(
        figsize=(20, 16), facecolor="none", edgecolor="black", linewidth=0.25
    )
    gdf.plot(ax=ax, **kwargs)
    ax.axis("off")
    if title:
        ax.set_title(title, fontsize=25)
    return ax

## Plot maps

In [ ]:
column_kwargs = {
    "Glacier": {
        "cmap": "coolwarm_r",
        "vmin": -5,
        "vmax": 5,
        "label": "Cumulative mass change (Gt)",
        "title": "Cumulative glacier mass change",
    },
    "Uncertainty": {
        "cmap": "coolwarm",
        "vmin": 0,
        "vmax": 5,
        "label": "Total mass change error (Gt)",
        "title": "Sum of glacier mass change error",
    },
    "Slope": {
        "cmap": "coolwarm_r",
        "vmin": -0.1,
        "vmax": +0.1,
        "label": "Mass change trend (Gt/yr)",
        "title": "Linear trends (slopes) of the mass changes of the glaciers.",
    },
    "Acceleration": {
        "cmap": "coolwarm_r",
        "vmin": -0.005,
        "vmax": +0.005,
        "label": "Total mass change error (Gt)",
        "title": "Quadratic trends (accelerations) of the mass changes of the glaciers.",
    },
}

for var_name, kwargs in column_kwargs.items():
    ax = plot_map(gdf, var_name, **kwargs)
    plt.show()